# Remoção de Outliers - Detecção e Tratamento

---

#### Neste notebook realizamos a remoção de outliers do dataset `clientes_limpeza.csv`.
#### As etapas incluem identificação, filtragem e tratamento de dados extremos para garantir qualidade nas análises futuras.

## 1. Leitura dos Dados

Importamos o dataset tratado anteriormente(`clientes_limpeza.csv`) e ajustamos a exibição.

- `scipy.stats` - A subbiblioteca `stats` do pacote `scipy` oferece funções estatísticas para análise de dados, ela é muito útil para detectar outliers, realizar testes estatísticos e calcular distribuições de probabilidade.

In [ ]:
import pandas as pd
from scipy import stats

pd.set_option('display.width', None)

df = pd.read_csv('../data/clientes_limpeza.csv')

## 2. Filtro Básico

Identificamos registros com idade superior a 100 anos como possível outlier.

In [ ]:
df_filtro_basico = df[df['idade'] > 100]
print('Filtro básico \n', df_filtro_basico[['nome', 'idade']])

## 3. Detecção de Outliers com Z-Score

- Z - Score mede o desvio padrão em relação à média.
- Valores com Z ≥ 3 são considerados outliers.

In [ ]:
df_zscore = df[(stats.zscore(df['idade']) < 3)]

## 4. Detecção de Outliers com IQR

- IQR = Intervalo Interquartil(Q3 - Q1)
- Outliers que estão fora dos limites:
    - `limite_baixo = Q1 - 1.5 * IQR`
    - `limite_alto = Q3 + 1.5 * IQR`

In [ ]:
Q1 = df['idade'].quantile(0.25)
Q3 = df['idade'].quantile(0.75)
IQR = Q3 - Q1

limite_baixo = Q1 - 1.5 * IQR
limite_alto = Q3 + 1.5 * IQR

print('Limites IQR: ', limite_baixo, limite_alto)

outliers_iqr = df[(df['idade'] < limite_baixo) | (df['idade'] > limite_alto)]
print('Outliers pelo IQR: \n', outliers_iqr)

## 5. Filtragem de Outliers

Removemos registros fora dos limites definidos.

In [ ]:
df_iqr = df[(df['idade'] >= limite_baixo) & (df['idade'] <= limite_alto)]

## 6. Definição de Faixa Válida

Aplicamos uma faixa de idade entre 1 e 100 anos para garantir consistência.

In [ ]:
limite_baixo = 1
limite_alto = 100
df = df[(df['idade'] >= limite_baixo) & (df['idade'] <= limite_alto)]

## 7. Validação de Endereços

Consideramos inválido qualquer endereço com menos de 3 linhas (ex: rua, bairro, cidade).

In [ ]:
df['endereco'] = df['endereco'].apply(lambda x: 'Endereço inválido' if len(x.split('\n')) < 3 else x)
print('Qtd registros com Endereço inválido: ', (df['endereco'] == 'Endereço inválido').sum())

## 8. Validação de Nomes

Nomes com mais de 50 caracteres são marcados como inválidos.

In [ ]:
df['nome'] = df['nome'].apply(lambda x: 'Nome inválido' if isinstance(x, str) and len(x) > 50 else x)
print('Qtd registros com nomes grandes: ', (df['nome'] == 'Nome inválido').sum())

## 9. Visualização Final

Exibimos os dados após tratamento de outliers e validações.

In [ ]:
print('Dados com Outliers tratados: \n', df)

## 10. Salvamento do Dataset Final

Exportamos o DataFrame limpo para `clientes_remocao_outliers.csv`.
- `index=False` - evita que o índice seja salvo no arquivo final.

In [ ]:
df.to_csv('clientes_remocao_outliers.csv', index=False)

#### Outras funções relevantes da `scipy.stats` (não utilizadas aqui, mas úteis para o futuro):

- `stats.ttest_ind()` – teste t para comparar médias entre dois grupos
- `stats.normaltest()` – verifica se os dados seguem distribuição normal
- `stats.pearsonr()` – calcula correlação de Pearson entre duas variáveis
- `stats.iqr()` – calcula o intervalo interquartil (alternativa ao cálculo manual do IQR)

#### Observações:

- A função `zscore()` ignora valores nulos automaticamente, mas é recomendável aplicar `.dropna()` antes para garantir precisão.
- O resultado é uma série de valores padronizados que podem ser usados para filtrar ou visualizar outliers.
- `scipy.stats` é compatível com arrays NumPy e colunas de DataFrames pandas.